# Google search try

In [ ]:
from googlesearch import search
import re
import pandas as pd

In [ ]:
companies = pd.read_csv('../data/processed/companies.csv')

In [ ]:
companies = companies['name'].tolist()

In [ ]:
# to search
found = set()

for idx,company in enumerate(companies):
    for j in search(company, tld="com"):
        print(idx)
        if re.search('wikipedia',j):
            found.add(company)
            break

In [ ]:
search

In [ ]:
import bs4 as bs
import urllib

comp = 'The Bank of East Asia Ltd'
url = 'https://en.wikipedia.org/wiki/'
print(urllib.parse.urlencode(comp))
# source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'html')

In [ ]:
for tag in soup.find_all('td'):
    if tag.get('class') == ['infobox-data', 'adr']:
        print(tag.text)

# Nominatim

In [1]:
from geopy.geocoders import Nominatim
import pandas as pd 
import os
import sys
import ssl
import numpy as np


sys.path.append('../')

In [2]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [3]:
geolocator = Nominatim(user_agent="my-application3", ssl_context=ssl_context, timeout=None)

In [5]:
# Random test
location = geolocator.geocode("175 5th Avenue NYC, United states")
location.latitude 

40.741059199999995

In [6]:
address = pd.read_parquet('../data/raw/addresses.parquet.gzip')
country_codes  = pd.read_excel('../data/raw/Country_codes.xlsx')

In [7]:
address.head()

,id,name,country,city_state_postal,location_street1
0,50472,"Infinity Pharmaceuticals, Inc.",USA,"Cambridge, MA 02138",1100 Massachusetts Avenue
1,282940,"Bozhon Precision Industry Technology Co., Ltd.",CHN,"Suzhou, Jiangsu 215222",666 Huxin West Road
2,313953,Quaker Houghton,USA,"Conshohocken, PA 19428-2380",One Quaker Park
3,1441,Kohl's Corp.,USA,"Menomonee Falls, WI 53051",N56 West17000 Ridgewood Drive
4,135666,"Sorrento Therapeutics, Inc.",USA,"San Diego, CA 92121",4955 Directors Place


In [8]:
address['completed'] = np.nan 

In [11]:
address = address.merge(country_codes, how='left', left_on='country', right_on='Alpha-3 code')

In [12]:
address

,id,name,country,city_state_postal,location_street1,completed,Country,Alpha-2 code,Alpha-3 code,Numeric
0,50472,"Infinity Pharmaceuticals, Inc.",USA,"Cambridge, MA 02138",1100 Massachusetts Avenue,NaN,United States of America (the),US,USA,840.0
1,282940,"Bozhon Precision Industry Technology Co., Ltd.",CHN,"Suzhou, Jiangsu 215222",666 Huxin West Road,NaN,China,CN,CHN,156.0
2,313953,Quaker Houghton,USA,"Conshohocken, PA 19428-2380",One Quaker Park,NaN,United States of America (the),US,USA,840.0
3,1441,Kohl's Corp.,USA,"Menomonee Falls, WI 53051",N56 West17000 Ridgewood Drive,NaN,United States of America (the),US,USA,840.0
4,135666,"Sorrento Therapeutics, Inc.",USA,"San Diego, CA 92121",4955 Directors Place,NaN,United States of America (the),US,USA,840.0
...,...,...,...,...,...,...,...,...,...,...
195920,371047,Zeneca Pharmaceuticals,USA,Delaware,None,NaN,United States of America (the),US,USA,840.0
195921,176574,Akzo Nobel Pakistan Ltd.,PAK,"Lahore, Punjab 54600","346, Ferozpur Road",NaN,Pakistan,PK,PAK,586.0
195922,94009,Emirates District Cooling Co. LLC,ARE,Dubai,Dubai Investments Park 1,NaN,United Arab Emirates (the),AE,ARE,784.0
195923,310366,Texas Health Resources,NAM,None,None,NaN,Namibia,NaN,NAM,516.0


In [10]:
address_new = address.loc[(address['completed'] != 1)&(address['country'] != 'NAM'))]

for id, rows in address_new.iterrows():

    print(id)

    if rows['completed'] != 1:

        if rows['city_state_postal'] is None:
            continue

        else:
             
            i = 0
            while i < len(rows['city_state_postal'].split(',')) :
                search_str = rows['city_state_postal'].split(',')[0:i].join(',') + rows['country']
                x = geolocator.geocode(search_str)
                if x is None:
                    i += 1
                    continue
                if x is not None:
                    lat = x.latitude
                    log = x.longitude
                    address.loc[id,'lat'] = lat
                    address.loc[id,'log'] = log
                    address.loc[id,'completed'] = 1
                    break
    
    if id % 1000 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')

6761
7505
8184
8192
8208
8251
8344
8353
8379
8409
8450
8463
8528
8586
8619
8630
8652
8710
8740
8832
8870
8905
8910
8948
8950
9017
9065
9084
9197
9201
9241
9273
9306
9377
9385
9401
9414
9425
9463
9468
9472
9491
9539
9557
9583
9595
9601
9605
9608
9623
9626
9628
9631
9632
9643
9673
9684
9689
9693
9696
9697
9709
9714
9718
9722
9724
9728
9730
9737
9745
9853
9855
9895
9950
9971
10014
10025
10086
10104
10108
10115
10126
10144
10169
10182
10197
10212
10224
10225
10230
10235
10238
10304
10407
10410
10439
10443
10447
10463
10469
10512
10534
10595
10616
10622
10692
10740
10742
10759
10762
10817
10876
10905
10994
11071
11075
11084
11154
11156
11197
11233
11240
11280
11297
11310
11311
11340
11345
11348
11367
11369
11385
11403
11414
11419
11484
11485
11500
11511
11513
11527
11592
11622
11631
11665
11673
11703
11708
11739
11751
11770
11775
11790
11791
11798
11803
11810
11843
11851
11858
11874
11875
11879
11895
11900
11907
11919
11944
11963
11995
12000
12047
12060
12070
12093
12125
12137
12152
12173
1

In [13]:
address = pd.read_parquet('../data/processed/addresses.parquet.gzip')

In [11]:
address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')

In [14]:
address.country.isna().sum()

46

In [15]:
country_codes 

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20
...,...,...,...,...
244,Western Sahara,EH,ESH,732
245,Yemen,YE,YEM,887
246,Zambia,ZM,ZMB,894
247,Zimbabwe,ZW,ZWE,716


In [16]:
address['city'] = address['city_state_postal'].str.split(',').str[0]

In [17]:
uni = address.groupby('city').agg(
    unique_country = pd.NamedAgg('country',lambda x: x.unique().tolist())
).reset_index()

In [21]:
mult = uni.loc[uni['unique_country'].apply(lambda x: len(x)) > 1]

In [19]:
mult_city = mult.explode('unique_country')

In [24]:
multi_cities = mult_city.merge(country_codes, left_on='unique_country', right_on='Alpha-3 code', how='left')

In [29]:
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Aachen'), 'Country'] = 'Germany'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Artarmon'), 'Country'] = 'Australia'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Ann Arbor'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Bangalore'), 'Country'] = 'India'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Boulder'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Cambridge'), 'Country'] = 'United Kingdom'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Lausanne'), 'Country'] = 'Switzerland'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Milwaukee'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Munich'), 'Country'] = 'Germany'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'New York'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Norwich'), 'Country'] = 'United Kingdom'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Oberhaching'), 'Country'] = 'Germany'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Paris'), 'Country'] = 'France'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Philadelphia'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Snoqualmie'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Southborough'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Stockholm 111 37'), 'Country'] = 'Sweden'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Towson'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Waltham'), 'Country'] = 'United States'
multi_cities.loc[(multi_cities['unique_country'] == 'NAM')&(multi_cities['city'] == 'Zurich 8002'), 'Country'] = 'Switzerland'

In [32]:
multi_cities

,city,unique_country,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Aachen,DEU,Germany,DE,DEU,276
1,Aachen,NAM,Germany,NaN,NAM,516
2,Aba,CHN,China,CN,CHN,156
3,Aba,NGA,Nigeria,NG,NGA,566
4,Abbotsford,AUS,Australia,AU,AUS,36
...,...,...,...,...,...,...
1687,Zurich 8002,NAM,Switzerland,NaN,NAM,516
1688,Zurich 8004,CHE,Switzerland,CH,CHE,756
1689,Zurich 8004,NOR,Norway,NO,NOR,578
1690,Zwijndrecht,NLD,Netherlands (the),NL,NLD,528


In [34]:
for id, row in multi_cities.iterrows():
    print(id)
    mask = (address['city'] == row['city'])&(address['country'] == row['unique_country'])

    x = geolocator.geocode(row['city']+ ", "+ row['Country'])
    
    if x is not None:

        lat = x.latitude
        log = x.longitude

        address.loc[mask,'lat'] = lat
        address.loc[mask,'log'] = log

    if x is None:

        address.loc[mask, 'lat'] = None
        address.loc[mask, 'log'] = None
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')
    

0
saved
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
saved
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
saved
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


In [23]:
address.to_csv('../data/processed/addresses.csv')